In [137]:
%matplotlib inline


import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import requests, json
import subprocess

In [138]:
#export
from nb_00 import *

### Download Hepg2 and K562 eclip and shRNA

Select appropriate filters at https://www.encodeproject.org/search/?type=Experiment&status=released&biosample_ontology.term_name=HepG2&assay_title=shRNA+RNA-seq&assay_title=eCLIP&biosample_ontology.term_name=K562

hit download. and grab this as first line from download file

https://www.encodeproject.org/metadata/type%3DExperiment%26status%3Dreleased%26biosample_ontology.term_name%3DHepG2%26assay_title%3DshRNA%2BRNA-seq%26assay_title%3DeCLIP%26biosample_ontology.term_name%3DK562/metadata.tsv

In [43]:
# Once you have the first line put behind !wget

!wget https://www.encodeproject.org/metadata/type%3DExperiment%26status%3Dreleased%26biosample_ontology.term_name%3DHepG2%26assay_title%3DshRNA%2BRNA-seq%26assay_title%3DeCLIP%26biosample_ontology.term_name%3DK562/metadata.tsv
makeFolders(["../samples_metadata"])
os.rename("metadata.tsv","../samples_metadata/metadata_K62_HEP_shRNA_eclip.tsv")    

--2019-08-12 10:26:20--  https://www.encodeproject.org/metadata/type%3DExperiment%26status%3Dreleased%26biosample_ontology.term_name%3DHepG2%26assay_title%3DshRNA%2BRNA-seq%26assay_title%3DeCLIP%26biosample_ontology.term_name%3DK562/metadata.tsv
Resolving www.encodeproject.org (www.encodeproject.org)... 34.211.244.144
Connecting to www.encodeproject.org (www.encodeproject.org)|34.211.244.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29207224 (28M) [text/tsv]
Saving to: ‘metadata.tsv’

metadata.tsv        100%[===================>]  27.85M  1.72MB/s    in 17s     

2019-08-12 10:26:45 (1.59 MB/s) - ‘metadata.tsv’ saved [29207224/29207224]



In [179]:
f1 = "../samples_metadata/metadata_K62_HEP_shRNA_eclip.tsv"
df = pd.read_csv(f1, sep="\t")
df = df[df["File format"]=="fastq"]
for col in list(df):
    if df[col].isnull().all(): # Remove nan columns
        del df[col]

df = df.rename(columns={"Assay": "assay", 
                   "Experiment target": "target",
                   'Library made from':"library",
                   "Biosample term name":"sample",
                   "Biosample organism":"organism",
                   "dbxrefs":"sra",
                   "File download URL":"url",
                    "File accession":"accession"
                  })        
        
# df["Biosample genetic modifications methods"] = df["Biosample genetic modifications targets"] \
#     .replace("/targets/Non-specific%20target%20","")\
#     .replace("/targets/","")\
#     .replace("-human","")        

df["assay"] = df["assay"].str.replace(" knockdown followed by RNA-seq","")
df["target"] = df["target"].str.replace("-human","").replace("Non-specific target control","Control")
        
df["sra"] = df["sra"].str.replace("SRA:","")    
df['library'] = df['library'].str.replace("polyadenylated mRNA","POLYA").str.replace("RNA","TOTAL")    
df["Paired end"] = df["Paired end"].astype(int).astype(str)
   
df["condition"] = df["target"]\
            +"_"+df["sample"]\
            +"_"+df["assay"]\
            +"_"+df['library']\
    
df["namePE"] = df["target"]\
            +"_"+df["sample"]\
            +"_"+df["assay"]\
            +"_"+df['library']\
            +"_"+df["sra"]\
            +"_"+df["Paired end"]

df["name"] = df["target"]\
            +"_"+df["sample"]\
            +"_"+df["assay"]\
            +"_"+df['library']\
            +"_"+df["sra"]\


In [180]:
# len(df)
# list(df)

In [194]:
def colToList(df,col):
    L = df[col].unique().tolist()
    return L

assays = colToList(df,"assay")
# mods = colToList(df,"Biosample genetic modifications methods")
samples = colToList(df,"sample")

urls = colToList(df,"url")
libraries = colToList(df,"library")
assays,samples,libraries
targets = colToList(df,"target")

def metaDataWrite(df,sample,assay,library):
    df = df[df["sample"]==sample]
    df = df[df["assay"]==assay]
    df = df[df["library"]==library]
    out = f"../samples_metadata/metadata/"
    if len(df) >1:
        makeFolders([out])
        df.to_csv(f'{out}/{sample}_{assay}_{library}.tsv',sep="\t",index=None)
        df["target"] = df["target"].drop_duplicates()
        df = df[["target"]].dropna(axis=0, how='all')
        df.to_csv(f'{out}/{sample}_{assay}_{library}_targets.tsv',sep="\t",index=None)
for sample in samples:
    for assay in assays:
        for library in libraries:
            metaDataWrite(df,sample,assay,library)

df.to_csv(f'{f1[:-4]}_cleaned.tsv',sep="\t", index=None)
samples

['HepG2', 'K562']

In [217]:
def targetDownload(df,target,assay,sample,library,control=False):
    if control:
        out = f"../samples_metadata/{assay}/{sample}/{library}/control/"
    else:    
        out = f"../samples_metadata/{assay}/{sample}/{library}/targets/"
    
    df = df[df["target"]==target]
    df = df[df["sample"]==sample]
    df = df[df["assay"]==assay]
    df = df[df["library"]==library]
    print(len(df))
    print(assay)
    if len(df) > 0:
        makeFolders([f"../samples_metadata/{assay}",f"../samples_metadata/{assay}/{sample}"])
        makeFolders([out])
        df.to_csv(f'{out}/{target}_{sample}_{assay}_{library}.tsv',sep="\t",index=None)
        df = df.dropna(axis=0,how="all",subset=["namePE"])
        print(assay)
        df_small = pd.DataFrame()
        for n,dfg in df.groupby("name"):
            dft = dfg.sort_values(by="namePE")
            dft["url2"] = dft["url"].iloc[1]
            dft["accession2"] = dft["accession"].iloc[1]
            dft = dft.iloc[0:1]   
            df_small = df_small.append(dft)

        df_small = df_small[["name","target","sample",
                       "assay","library","condition","url","accession",
                      "url2","accession2"]]    
        df_small.to_csv(f'{out}/{target}_{sample}_{assay}_{library}_download.tsv',sep="\t",index=None)

        df_small = df_small[["name","target","sample",
                       "assay","library","condition"]]    
        df_small.to_csv(f'{out}/{target}_{sample}_{assay}_{library}_small.tsv',sep="\t",index=None)

        
        
    return df_small



In [218]:
#FUS, MATR3, STAU1, TIA1

targets = ["TARDBP","FUS"]
sample = "K562"
library = "POLYA"
assay = "shRNA"


libraries = ["POLYA"]

for t in targets:
    for library in libraries:
        df1 = targetDownload(df,target="Control",
                                sample=sample,
                                assay=assay,
                                library=library,
                            control=True)

        df2 = targetDownload(df, target=t,
                                sample=sample,
                                assay=assay,
                                library=library)


# dfC = targetDownload(df,"Control")

96
shRNA
shRNA
4
shRNA
shRNA
96
shRNA
shRNA
4
shRNA
shRNA


In [197]:
df3 = df1.iloc[0:4]
dfglist = []
for n,dfg in df3.groupby("name"):
    print("******")
    print(n)
    dfglist.append(dfg)

******
Control_K562_shRNA_POLYA_SRR4422597
******
Control_K562_shRNA_POLYA_SRR4422598


In [193]:
dft = dfglist[0]

dft

,accession,File format,Output type,Experiment accession,assay,Biosample term id,sample,Biosample type,organism,Biosample genetic modifications methods,...,url,Platform,Controlled by,File Status,s3_uri,Audit WARNING,Audit INTERNAL_ACTION,condition,namePE,name
3,ENCFF565XPN,fastq,reads,ENCSR118VQR,shRNA,EFO:0001187,HepG2,cell line,Homo sapiens,RNAi,...,https://www.encodeproject.org/files/ENCFF565XP...,Illumina HiSeq 2000,"/files/ENCFF006BJQ/, /files/ENCFF366MJN/",released,s3://encode-public/2015/08/13/4fca079a-e471-4b...,missing biosample characterization,NaN,RPS3A_HepG2_shRNA_POLYA,RPS3A_HepG2_shRNA_POLYA_SRR4421372_1,RPS3A_HepG2_shRNA_POLYA_SRR4421372


In [34]:
# Use to download one file with python

def downloadRenameFastq(df):
    for x in range(len(df)):
        print("downloading ",x)
        url = df["url"].iloc[x]
        accession = df["accessiongz"].iloc[x]
        name = df["namegz"].iloc[x]
        subprocess.run(f"wget {url}",shell=True)
        os.rename(accession,name)
        print("done downloading",x)
    
# downloadRenameFastq(dfC1)